In [2]:
from PIL import Image, ImageFilter, ImageEnhance

def create_halation(image_path, output_path, blur_radius=10, brightness_threshold=150, tint_color=(255, 100, 100)):
    """
    Creates a halation effect on an image.

    Args:
        image_path (str): Path to the input image.
        output_path (str): Path to save the output image with halation.
        blur_radius (int): The radius of the Gaussian blur applied to the bright areas.
        brightness_threshold (int): Pixels brighter than this value will contribute to halation.
        tint_color (tuple): RGB tuple for the color tint of the halation (e.g., red for filmic look).
    """
    try:
        original_image = Image.open(image_path).convert("RGB")
    except FileNotFoundError:
        print(f"Error: Image not found at {image_path}")
        return

    # Create a mask for bright areas
    bright_mask = Image.new("L", original_image.size, 0)
    pixels = original_image.load()
    mask_pixels = bright_mask.load()

    for y in range(original_image.height):
        for x in range(original_image.width):
            r, g, b = pixels[x, y]
            brightness = (r + g + b) // 3  # Simple brightness calculation
            if brightness > brightness_threshold:
                mask_pixels[x, y] = 255  # Mark as bright

    # Apply Gaussian blur to the mask
    blurred_mask = bright_mask.filter(ImageFilter.GaussianBlur(radius=blur_radius))

    # Create a colored layer for halation
    halation_layer = Image.new("RGB", original_image.size, tint_color)
    
    # Blend the halation layer using the blurred mask
    # The 'alpha_composite' function requires images with an alpha channel.
    # So, we convert the halation layer to RGBA and use the blurred mask as the alpha channel.
    halation_layer_with_alpha = Image.new("RGBA", original_image.size)
    halation_layer_pixels = halation_layer.load()
    halation_layer_with_alpha_pixels = halation_layer_with_alpha.load()
    blurred_mask_pixels = blurred_mask.load()

    for y in range(original_image.height):
        for x in range(original_image.width):
            r, g, b = halation_layer_pixels[x, y]
            alpha = blurred_mask_pixels[x, y]
            halation_layer_with_alpha_pixels[x, y] = (r, g, b, alpha)

    # Blend the halation layer with the original image
    final_image = Image.alpha_composite(original_image.convert("RGBA"), halation_layer_with_alpha)
    return final_image

In [ ]:
path="/Users/alayshah/Desktop/DSCF2897.JPG"
create_halation(image_path=path, output_path="")